In [ ]:
import pandas as pd
import numpy as np
import string
import pickle
import io
import torch
from sentence_transformers import SentenceTransformer

metadata = pd.read_csv("Movies_data(3200).csv")

with open('encodings_description.pickle','rb') as file:
    encoded_data = pickle.load(file)

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

with open('model.pickle','rb') as file:
    #model = pickle.load(file)
    model = CPU_Unpickler(file).load()

def find_similar_items(query: str, metadata: pd.DataFrame, model: SentenceTransformer, encodings: np.array, top_n: int = 5):
    query_vector = model.encode([query])
    # print(query_vector)
    similarity = np.dot(encodings,query_vector.T)
    # print(similarity)
    top_items = similarity.flatten().argsort()[-top_n:][::-1]
    # print(top_items)
    # return metadata['asin'].iloc[top_items]
    return list(metadata['movieId'].iloc[top_items].values)

top_n = 10
query = "adventure genre with title jumanji"

top_items_mID = find_similar_items(query=query, metadata=metadata, model=model, encodings=encoded_data, top_n=top_n)